## The .prm file

For this module, you will need the parameter file 'mantle-flow.prm' which is available in the current directory. All the input parameters in this file can be loaded as a dictionary using the python script, `aspect_prmf.py` available [here](https://github.com/mgouiza/aspect-prmf).

This file  provides the numerical parameters to run the simulation: convection in a 2D box over time as it is heated from below.

Below, we will walk you through the key parameters used to formulate this model.

ASPECT uses SI units: meter, kg, seconds/years.

<div class="alert alert-block alert-info">
<b>NOTE:</b> The parameter names and their values are case-sensitive. This means that using `Function` instead of `function` will throw an error in ASPECT.
</div>

### Global Parameters

These parameter values are loaded in the `main` function in ASPECT to describe the simulation parameters that determine the overall behavior of the simulation such as the time duration and the output directory to store the solution files.

```
set Dimension                              = 2
set Use years in output instead of seconds = true
set End time                               = 5e10
set Output directory                       = output-mantle-flow
```


### Geometry Model
ASPECT has many built-in geometry models, e.g., "box" and "spherical shell" models. Box models can be rectangular (2D) or cuboid (3D). The snippet below demonstrates how to set the geometry model in the input file:

```
subsection Geometry model
    set Model name = box
    subsection Box
        set X extent = 4.2e6
        set Y extent = 3e6
    end
end
```


### Initial Temperature
Initial condition models specify the initial temperature. Modeling the initial temperature as a function lets us specify the initial temperature as a formula, with user defined constants. In this example, we are specifying a sinusoidal perturbation to a linear temperature profile. This can be expressed as:

$ T(x,y) = T_{top} + (T_{bottom} - T_{top}) (1 - \frac{y}{d} -pcos(\frac{k{\pi}x}{L})sin(\frac{{\pi}y}{d}))$,

and written in the prm file as:

```
subsection Initial temperature model
    set Model name = function
    
    subsection Function
        set Variable names = x,y
        set Function constants = p=-0.01, L=4.2e6, D=3e6,pi=3.1415926536, k=1, T_top=273, T_bottom=3600
        set Function expression = T_top + (T_bottom-T_top)*(1-(y/D)-p*cos(k*pi*x/L)*sin(pi*y/D)))
    end
end
```


### Temperature at the Boundary
In this model, the temperature is fixed at 3600K at the bottom of the box and 273K at the top of the box. Depending on the model, left and right temperature options can be similarly specified. In 3D models, you can also specify the `back` and `front` temperatures. When boundaries are unspecified, the default is no heat flux (i.e., an insulated boundary).

Specify the top and bottom temperature using the following section:

```
subsection Boundary temperature model
    set Fixed boundary indicators = top, bottom
    set Model name = box
    subsection Box
        set Bottom temperature = 3600
        set Top temperature = 273
    end
end
```

### Velocity at the Boundary
Velocity at the boundaries can be specified by:
- Prescribing the velocity at the boundaries manually
- Choosing prescribed tractions
- Choosing "free slip" boundaries

The section below specifies free slip boundaries on all sides:

```
subsection Boundary velocity model
    set Zero velocity boundary indicators =
    set Tangential velocity boundary indicators = left, right, top, bottom
end
```

### Discretization
The initial global refinement specifies the mesh “grid spacing”. An additional level of refinement increases the number of cells by a factor of 4 in 2D. The model can also be refined at specific regions using "Initial adaptive refinement" criteria. In this file, we only use global refinement as:

```
subsection Mesh refinement
    set Initial global refinement = 3
    set Initial adaptive refinement = 0
    set Time steps between mesh refinement = 0
end
```

### Material Model
This section describes the material properties of the model, such as density, viscosity, thermal conductivity etc. ASPECT provides various built in material models from which the user can choose. In this file, `simpler` model is used and, as specified here, has constant value for all material properties, except density which depends on temperature as:

$\rho (p, T) = \rho_0 (1 - \alpha (T - T_0))$,

where, $\rho_0, \alpha, T$, represent the reference density, thermal expansivity, and temperature, respectively. The variations in density of the material lead to the resultant buoyancy forces needed to drive the convective flow.

More details are available in the ASPECT manual on the formulation of the material model.

```
subsection Material model
  set Model name = simpler

  subsection Simpler model
    set Reference density             = 3300
    set Reference specific heat       = 1250
    set Reference temperature         = 293.
    set Thermal conductivity          = 4.7
    set Thermal expansion coefficient = 2e-5
    set Viscosity                     = 5.10452e24
  end

end
```

### Postprocessing
The postprocessing section of the input file tells ASPECT how to analyze the simulation data. The section below specifies heat flow and other statistics that are output with the solution. In this case, graphical output is generated every $10^7$ years.

```
subsection Postprocess
    set List of postprocessors = velocity statistics, temperature statistics, heat flux statistics, visualization, basic statistics
    subsection Visualization
        set Time between graphical output = 1e7
        set Output format = vtu
        set List of output variables = material properties
    end
end
```

### Further reading
For futher information on the parameter description in ASPECT, please refer to the online [ASPECT manual](https://aspect-documentation.readthedocs.io/en/latest/index.html).

 &nbsp;<div style="text-align: right">  
    &rarr; <b>NEXT: [Plate tectonics and basic equations](./2_plate_tectonics_and_basic_equations.ipynb) </b> <a href=""></a> &nbsp;&nbsp;
     <img src="../assets/education-gem-notebooks_icon.png" alt="icon"  style="width:4%">
  </div>